<a href="https://colab.research.google.com/github/KoMurase/movie_research_2/blob/master/Preprocessing_Title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#元データ使う場合
#!cp -r '/content/drive/MyDrive/Movies/data/mojo_japan/' '/content/'
#!cp -r '/content/drive/MyDrive/Movies/data/metadata_filmarks/' '/content/'

In [2]:
#前処理済みデータを使う場合
!cp -r '/content/drive/MyDrive/Movies/preprocessed/' '/content/'

In [3]:
!pip install texthero
!pip install textdistance

     |████████████████████████████████| 1.4MB 17.1MB/s 
     |████████████████████████████████| 245kB 28.1MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434677 sha256=1ba85846f461e0df8b8604087be2bf94e64489889c02cd0bbcca4b8db608d4e0
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [4]:
!pip install japanize_matplotlib
import pandas as pd
import codecs 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import japanize_matplotlib

import warnings

import os
from tqdm import tqdm

     |████████████████████████████████| 4.1MB 10.9MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-cp36-none-any.whl size=4120276 sha256=9792a748501277f20ed3efd6132f78c56a6f6886d3b1d4799817cd8c04fbab86
  Stored in directory: /root/.cache/pip/wheels/b7/d9/a2/f907d50b32a2d2008ce5d691d30fb6569c2c93eefcfde55202
Successfully built japanize-matplotlib


In [5]:
def read_data(dir, encode_type):
  output=pd.DataFrame()
  print(dir)
  files=os.listdir(dir)
  print(files)

  for file_ in files:
    if '2016' or '2017' or '2018' or '2019' in file_: #収集できているファイル


      path = dir+file_
      print(path)
      with codecs.open(path, "r", encode_type, "ignore") as file:
          d = pd.read_csv(file, delimiter=",")
      
      output = pd.concat([output, d], axis=0)

  return output

In [6]:
prepro_dir='/content/preprocessed/'
train = pd.read_csv(prepro_dir+'train_2016-2018.csv')
test = pd.read_csv(prepro_dir+'test_2019.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5628) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df = pd.concat([train, test], axis=0).reset_index(drop=True)

In [8]:
"""metadata_dir='/content/metadata_filmarks/'
mojo_dir='/content/mojo_japan/'

data = read_data(metadata_dir, encode_type='utf-8')
mdata = read_data(mojo_dir, encode_type='shift-jis')
mdata = mdata.rename(columns={'Filmarks URL':'URL'})
df = data.merge(mdata, how='left',on='URL').reset_index(drop=True)
drop_cols=[
           'url','Release Date','タイトル(日本名)','タイトル(英名)', 
           'header情報',  'あらすじ'
]
df = df.drop(drop_cols, axis=1) """

"metadata_dir='/content/metadata_filmarks/'\nmojo_dir='/content/mojo_japan/'\n\ndata = read_data(metadata_dir, encode_type='utf-8')\nmdata = read_data(mojo_dir, encode_type='shift-jis')\nmdata = mdata.rename(columns={'Filmarks URL':'URL'})\ndf = data.merge(mdata, how='left',on='URL').reset_index(drop=True)\ndrop_cols=[\n           'url','Release Date','タイトル(日本名)','タイトル(英名)', \n           'header情報',  'あらすじ'\n]\ndf = df.drop(drop_cols, axis=1) "

In [9]:
df=df[df['Title'].duplicated()==False]
df.shape

(673, 9017)

In [10]:
from nltk.util import ngrams
import texthero as hero
from texthero import preprocessing

def line_ngram(line, n=2):
  words = [w for w in line.split(' ') if len(w) != 0]
  return list(ngrams(words, n))

def clearn_by_hero(x):
    custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   , preprocessing.remove_whitespace
                   , preprocessing.remove_stopwords
                  ]
    return hero.clean(x, custom_pipeline)
    
def create_n_gram(x, flg=False, n=3): 
    if flg==True:
      x = clearn_by_hero(x)
    x = pd.Series(x).map(lambda r: line_ngram(r, n=n))
    return x

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
from textdistance import jaro

In [12]:
custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   , preprocessing.remove_whitespace
                   , preprocessing.remove_stopwords
                  ]

In [13]:
df['Title'] = hero.clean(df['Title'], custom_pipeline)
table = pd.DataFrame(df['Title'])
title = df['Title'].values
for t in tqdm(title):
  table[f'{t}']=table['Title'].map(lambda x: jaro(t, x))

100%|██████████| 673/673 [00:20<00:00, 33.48it/s]


In [14]:
table

,Title,bohemian rhapsody,ralph breaks internet,secret life pets,dragon ball super broly,star born,banana time night,wonderful family wife life,fantastic beasts crimes grindelwald,invisible witness,kamen rider heisei generations forever,grinch,dr seuss grinch,yo kai watch forever friends,nutcracker four realms,pad man,venom,paul apostle christ,searching,christopher robin,beauty beast,despicable,pirates caribbean dead men tell tales,name,moana,sing,star wars episode viii last jedi,detective conan crimson love letter,doraemon great adventure antarctic kachi kochi,la la land,fate furious,gintama live action movie,want eat pancreas,pokteumon movie choose,mary witch flower,spider man homecoming,memoirs murderer,sekigahara,resident evil final chapter,mumon land stealth,...,yo kai watch jam movie yo kai academy cat hero,suspiria,mortal engines,dance,documentary nogizaka,psycho pass sinners system case crime punishment,prince legend,city hunter,promised land,listen universe,taxi,simple favor,black school rules,child play,ninkyo gakuen,train dragon hidden world,boy erased,front runner,angel fallen,brightburn,ai uta yakusoku nakuhito,king prism shiny seven stars,family strangers,chia danshi,leaving scene,kabaneri iron fortress battle unato,happy death day 2u,glass castle,love,life,fighting family,blindspotting,wildlife,marrowbone,welcome marwen,beautiful boy,days entebbe,first purge,human flow,dark hall
0,bohemian rhapsody,1.000000,0.564171,0.450980,0.617531,0.466132,0.450514,0.507190,0.496604,0.395425,0.535002,0.518908,0.473564,0.548459,0.430196,0.536134,0.511765,0.511656,0.506536,0.538126,0.593838,0.449554,0.519474,0.511765,0.698039,0.539216,0.497089,0.532569,0.465523,0.531373,0.483287,0.467760,0.540196,0.555730,0.444398,0.541550,0.555011,0.484314,0.528457,0.511656,...,0.498320,0.406046,0.575941,0.518908,0.568833,0.519356,0.483754,0.521687,0.634174,0.513072,0.372549,0.507218,0.513344,0.577243,0.426244,0.511021,0.673529,0.347662,0.492911,0.464706,0.529412,0.465104,0.522059,0.649287,0.538282,0.491522,0.555011,0.514329,0.447712,0.481046,0.457516,0.426244,0.289216,0.590196,0.483287,0.602456,0.347662,0.439542,0.484314,0.516221
2,ralph breaks internet,0.564171,1.000000,0.626263,0.641786,0.594276,0.594517,0.501010,0.606807,0.536700,0.634326,0.410534,0.620160,0.605844,0.518182,0.580519,0.415152,0.644406,0.590909,0.564171,0.556277,0.594276,0.607912,0.467677,0.496970,0.000000,0.556566,0.570130,0.541035,0.568687,0.507323,0.550117,0.604040,0.555758,0.518182,0.553391,0.643098,0.513131,0.620400,0.531818,...,0.453186,0.484091,0.689610,0.501082,0.530109,0.564765,0.572872,0.494949,0.589610,0.667112,0.431818,0.534674,0.536700,0.494949,0.470629,0.525882,0.577489,0.534674,0.534674,0.509091,0.551515,0.621909,0.564171,0.556277,0.670996,0.660798,0.562290,0.534674,0.434343,0.467677,0.481411,0.476024,0.392677,0.457576,0.507323,0.411422,0.607925,0.490981,0.360606,0.571970
4,secret life pets,0.450980,0.626263,1.000000,0.472424,0.570707,0.525132,0.538047,0.612913,0.555556,0.559904,0.514550,0.465939,0.651913,0.592795,0.399471,0.418519,0.568855,0.544444,0.513344,0.546958,0.532227,0.625000,0.503704,0.000000,0.537037,0.591150,0.598413,0.522517,0.377778,0.640741,0.526548,0.618519,0.551852,0.568855,0.478836,0.555556,0.457407,0.715939,0.501852,...,0.446581,0.402778,0.546958,0.514550,0.578348,0.575817,0.640212,0.532227,0.486772,0.631511,0.537037,0.561610,0.497354,0.515152,0.354701,0.490102,0.525926,0.487464,0.487179,0.377778,0.546481,0.554598,0.565359,0.361953,0.566138,0.556980,0.518519,0.561610,0.546296,0.625926,0.504980,0.487179,0.490741,0.270370,0.575926,0.487179,0.547212,0.620370,0.377778,0.444108
5,secret life pets,0.450980,0.626263,1.000000,0.472424,0.570707,0.525132,0.538047,0.612913,0.555556,0.559904,0.514550,0.465939,0.651913,0.592795,0.399471,0.418519,0.568855,0.544444,0.513344,0.546958,0.532227,0.625000,0.503704,0.000000,0.537037,0.591150,0.598413,0.522517,0.377778,0.640741,0.526548,0.618519,0.551852,0.568855,0.478836,0.555556,0.457407,0.715939,0.501852,...,0.

In [15]:
for c in table.columns:
  if c!='Title':
    table[c]=table[c].astype(float)

In [16]:
count=0
movie_series=[]
for c in table.columns:
  
  if c!='Title':
    #print(table[table[c]==1]['Title'].values[0])
    result=table[(table[c]>0.77)&(table[c]!=1)]['Title'].values
    if len(result) >0:
      print(f'-{c}-----')
      print(table[(table[c]>0.77)&(table[c]!=1)]['Title'].values)
    
    movie_series.extend(result)

-  wonderful family  wife  life-----
['  wonderful family' '  wonderful family']
-fantastic beasts  crimes  grindelwald-----
['fantastic beasts    find ']
-kamen rider heisei generations forever-----
['kamen rider heisei generations final build ex aid  legend riders'
 'kamen rider ex aid true ending'
 'kamen rider heisei generations dr pac man vs ex aid ghost  legend rider'
 'kamen rider reiwa  first generation']
-yo kai watch forever friends-----
['yo kai watch']
-sing-----
['sing street' 'sing  life']
-star wars episode viii  last jedi-----
['star wars episode vii  force awakens'
 'star wars episode ix  rise  skywalker']
-detective conan crimson love letter-----
['detective conan zero  enforcer' 'detective conan  darkest nightmare']
-pokteumon  movie  choose -----
['pokteumon  movie  power  us'
 'pokteumon  movie volcanion   mechanical marvel']
-spider man homecoming-----
['spider man far  home']
-mumon  land  stealth-----
['man   gold medal']
-cars-----
['mars' 'carol']
-fantastic b

In [17]:
movie_series=sorted(list(set(movie_series)))
movie_series

['   ',
 '   hero',
 '   return',
 '  brave',
 '  breathe',
 '  corner   world',
 '  heart   sea',
 '  love',
 '  money   world',
 '  paris',
 '  rain',
 '  vanish',
 '  wonderful family',
 '  wonderful family  wife  life',
 '  x',
 ' 100th love  ',
 ' comes',
 ' commuter',
 ' dark tower',
 ' departures',
 ' dog  journey',
 ' dog  purpose',
 ' drive',
 ' equalizer',
 ' founder',
 ' girl   train',
 ' glass castle',
 ' great wall',
 ' greatest showman',
 ' hero academia heroes rising',
 ' hero academia two heroes',
 ' last white witch',
 ' last witch hunter',
 ' lie  april',
 ' lion king',
 ' love',
 ' magnificent nine',
 ' magnificent seven',
 ' man  knew infinity',
 ' martian',
 ' predator',
 ' shallows',
 ' silent voice  movie',
 ' teacher',
 ' teacher  love',
 ' walk',
 ' wall',
 ' zero kara hajimeru isekai seikatsu  zero kara hajimeru isekai seikatsu memory snow',
 ' zero kara hajimeru isekai seikatsu hyouketsu  kizuna',
 'american assassin',
 'american made',
 'american renegades',

In [18]:
word_counter_en = {}
for title in movie_series:
  for word in title.split(' '):
    # dict.get(key, 0)はdictにkeyがあればdict[key]を、なければ0を返すメソッドです
    word_counter_en[word] = word_counter_en.get(word, 0) + 1

In [19]:
word_counter_en

{'': 181,
 '100th': 1,
 'academia': 2,
 'ahead': 1,
 'aid': 3,
 'alien': 1,
 'america': 1,
 'american': 3,
 'anemone': 1,
 'april': 1,
 'assassin': 1,
 'awakens': 1,
 'baby': 1,
 'battleship': 3,
 'beasts': 2,
 'birth': 1,
 'blue': 1,
 'bond': 1,
 'boy': 1,
 'brave': 1,
 'breathe': 1,
 'build': 2,
 'butterfly': 1,
 'captain': 2,
 'carol': 1,
 'cars': 1,
 'castle': 1,
 'ch': 1,
 'chan': 2,
 'chapter': 4,
 'chihayafuru': 3,
 'choose': 1,
 'city': 2,
 'civil': 1,
 'clash': 1,
 'coco': 1,
 'coda': 1,
 'code': 2,
 'comes': 1,
 'comet': 1,
 'commuter': 1,
 'conan': 4,
 'corner': 1,
 'crayon': 2,
 'crimes': 1,
 'crimson': 1,
 'cure': 3,
 'dark': 1,
 'darkest': 2,
 'das': 2,
 'dawn': 1,
 'day': 1,
 'departure': 1,
 'departures': 1,
 'der': 1,
 'destiny': 1,
 'detective': 4,
 'detonation': 1,
 'diner': 1,
 'dog': 2,
 'doraemon': 2,
 'dr': 1,
 'drive': 1,
 'driver': 1,
 'eater': 1,
 'end': 1,
 'ending': 1,
 'enforcer': 1,
 'episode': 3,
 'equalizer': 1,
 'equals': 1,
 'erased': 2,
 'eureka': 2,


In [20]:
def prepro_title(text):
  text = text.replace(':','')\
             .replace('：','')\
             .replace('劇場版','')\
             .replace('映画','')\
             .replace('\u3000',' ')\
             .replace('　',' ')\
             .replace('?',' ')\
             .replace('？',' ')\
             .replace('/','')\
             .replace('〜','')\
             .replace('ファンタスティック・ビーストと','ファンタスティック・ビースト ')\
             .replace('スパイダーマンスパイダーバース','スパイダーマン スパイダーバース')\
             .replace('スパイダーマンファー・フロム・ホーム','スパイダーマン ファー・フロム・ホーム')
  return text

In [21]:
df['Title(ja)'] = df['Title(ja)'].dropna().apply(prepro_title)
table_ja = pd.DataFrame(df['Title(ja)'].dropna())

title = df['Title(ja)'].dropna().values
for t in tqdm(title):
  table_ja[f'{t}']=table_ja['Title(ja)'].map(lambda x: jaro(t, x))
  table_ja[f'{t}']=table_ja[f'{t}'].astype(float)

100%|██████████| 672/672 [00:15<00:00, 43.07it/s]


In [22]:
table_ja

,Title(ja),ボヘミアン・ラプソディ,シュガー・ラッシュオンライン,ペット,ドラゴンボール超 ブロリー,アリー スター誕生,こんな夜更けにバナナかよ 愛しき実話,ファンタスティック・ビースト 黒い魔法使いの誕生,インビジブル・ウィットネス 見えない目撃者,平成仮面ライダー20作記念 仮面ライダー平成ジェネレーションズFOREVER,グリンチ,妖怪ウォッチ FOREVER FRIENDS,くるみ割り人形と秘密の王国,パッドマン 5億人の女性を救った男,ヴェノム,パウロ 愛と赦しの物語,search／サーチ,プーと大人になった僕,美女と野獣,怪盗グルーのミニオン危機一発,パイレーツ・オブ・カリビアン最後の海賊,君の名は,モアナと伝説の海,シング,スター・ウォーズ最後のジェダイ,名探偵コナン から紅の恋歌,ドラえもん のび太の南極カチコチ大冒険,ラ・ラ・ランド,ワイルド・スピード ICE BREAK,銀魂,君の膵臓を食べたい,ポケットモンスター キミにきめた!,メアリと魔女の花,スパイダーマン ホームカミング,22年目の告白 私が殺人犯です,関ヶ原,バイオハザードザ・ファイナル,忍びの国,昼顔,ソードアートオンライン-オーディナル・スケール,...,妖怪学園Y 猫はHEROになれるか,サスペリア,移動都市／モータル・エンジン,ダンスウィズミー,いつのまにか、ここにいる Documentary of 乃木坂46,PSYCHO-PASS サイコパス Sinners of the System Case.1「罪と罰」,PRINCE OF LEGEND,シティーハンター THE MOVIE 史上最香のミッション,楽園,蜜蜂と遠雷,TAXi ダイヤモンド・ミッション,シンプル・フェイバー,ブラック校則,チャイルド・プレイ,任侠学園,ヒックとドラゴン 聖地への冒険,ある少年の告白,フロントランナー,エンド・オブ・ステイツ,ブライトバーン恐怖の拡散者,愛唄 ―約束のナクヒト―,KING OF PRISM -Shiny Seven Stars- III レオ×ユウ×アレク,閉鎖病棟ーそれぞれの朝ー,チア男子!!,轢き逃げ −最高の最悪な日−,甲鉄城のカバネリ 海門決戦,ハッピー・デス・デイ 2U,ガラスの城の約束,愛がなんだ,ライフ・イットセルフ 未来に続く物語,ファイティング・ファミリー,ブラインドスポッティング,ワイルドライフ,マローボーン家の掟,マーウェン,ビューティフル・ボーイ,エンテベ空港の7日間,パージエクスペリメント,ヒューマン・フロー 大地漂流,ダーク・スクール
0,ボヘミアン・ラプソディ,1.000000,0.384560,0.000000,0.390054,0.400673,0.000000,0.354798,0.471861,0.372408,0.446970,0.000000,0.000000,0.383244,0.0,0.000000,0.000000,0.000000,0.0,0.441558,0.381180,0.0,0.405303,0.474747,0.385859,0.389277,0.381180,0.455988,0.381180,0.0,0.0,0.383244,0.405303,0.385859,0.000000,0.0,0.387446,0.00,0.0,0.490997,...,0.000000,0.430303,0.387446,0.310606,0.000000,0.000000,0.000000,0.250261,0.0,0.0,0.433155,0.460606,0.000000,0.468013,0.0,0.271717,0.000000,0.477273,0.454545,0.278555,0.000000,0.000000,0.000000,0.419192,0.000000,0.000000,0.445221,0.000000,0.0,0.264310,0.445221,0.396465,0.411255,0.468013,0.430303,0.393939,0.396970,0.000000,0.441558,0.405303
2,シュガー・ラッシュオンライン,0.384560,1.000000,0.468254,0.265568,0.394180,0.000000,0.317460,0.341270,0.362907,0.000000,0.371636,0.000000,0.420168,0.0,0.000000,0.390476,0.390476,0.0,0.476190,0.498747,0.0,0.000000,0.468254,0.471429,0.382784,0.374687,0.436508,0.249373,0.0,0.0,0.352474,0.000000,0.471429,0.000000,0.0,0.476190,0.00,0.0,0.507591,...,0.000000,0.000000,0.523810,0.398810,0.000000,0.363679,0.000000,0.439245,0.0,0.0,0.482726,0.478571,0.492063,0.455026,0.0,0.360317,0.000000,0.418651,0.495671,0.370574,0.000000,0.000000,0.384921,0.000000,0.000000,0.000000,0.447802,0.464286,0.0,0.335979,0.265568,0.488095,0.404762,0.455026,0.514286,0.549784,0.000000,0.441558,0.440476,0.464286
4,ペット,0.000000,0.468254,1.000000,0.000000,0.000000,0.000000,0.583333,0.587302,0.000000,0.000000,0.458937,0.000000,0.464052,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.594771,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.458937,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.500000,0.000000,0.0,0.466667,0.000000,0.486111,0.000000,0.470085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.470085,0.000000,0.0,0.592593,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,ペット,0.000000,0.468254,1.000000,0.000000,0.000000,0.000000,0.583333,0.587302,0.000000,0.000000,0.458937,0.000000,0.464052,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.594771,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.458937,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.500000,0.000000,0.0,0.466667,0.000000,0.486111,0.000000,0.470085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.470085,0.000000,0.0,0.592593,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,ドラゴンボール超 ブロリー,0.390054,0.265568,0.000000,1.000000,0.458689,0.377493,0.451923,0.416056,0.470985,0.442308,0.373467,0.000000,0.469080,0.0,0.389277,0.392308,0.392308,0.0,0.370574,0.462888,0.0,0.000000,0.470085,0.429060,0.435897,0.462

In [23]:
movie_series_ja=[]

for c in table_ja.columns:
  
  if c!='Title(ja)':
    #print(table[table[c]==1]['Title'].values[0])
    result=table_ja[(table_ja[c]>0.70)&(table_ja[c]!=1)]['Title(ja)'].values
    if len(result) >0:
      print(f'-{c}-----')
      print(table_ja[(table_ja[c]>0.70)&(table_ja[c]!=1)]['Title(ja)'].values)
    
    movie_series_ja.extend(result)

-アリー スター誕生-----
['アス']
-ファンタスティック・ビースト 黒い魔法使いの誕生-----
['ファンタスティック・ビースト 魔法使いの旅']
-グリンチ-----
['グリーンブック']
-シング-----
['シング・ストリート 未来へのうた']
-スター・ウォーズ最後のジェダイ-----
['スター・ウォーズ／フォースの覚醒']
-名探偵コナン から紅の恋歌-----
['名探偵コナン ゼロの執行人' '名探偵コナン 純黒の悪夢' '名探偵コナン 紺青の拳']
-ドラえもん のび太の南極カチコチ大冒険-----
['ドラえもん のび太の宝島' 'ドラえもん 新・のび太の日本誕生' 'ドラえもん のび太の月面探査記']
-ポケットモンスター キミにきめた!-----
['ポケットモンスター みんなの物語']
-スパイダーマン ホームカミング-----
['スパイダーマン ファー・フロム・ホーム' 'スパイダーマン スパイダーバース']
-IT／イット “それ”が見えたら、終わり。-----
['IT／イット THE END “それ”が見えたら、終わり。']
-ローグ・ワンスター・ウォーズ・ストーリー-----
['ハン・ソロ／スター・ウォーズ・ストーリー']
-トランスフォーマー／最後の騎士王-----
['テラフォーマーズ']
-ファンタスティック・ビースト 魔法使いの旅-----
['ファンタスティック・ビースト 黒い魔法使いの誕生']
- Fatestay night Heaven's Feel I. presage flower-----
[" Fatestay night Heaven's Feel II. lost butterfly"]
- 妖怪ウォッチ 空飛ぶクジラとダブル世界の大冒険だニャン!-----
[' 妖怪ウォッチ 誕生の秘密だニャン！']
-HiGH&LOW THE MOVIE2  END OF SKY-----
['HiGH&LOW THE RED RAIN' 'HiGH&LOW THE MOVIE' 'HiGH&LOW THE WORST']
-僕のワンダフル・ライフ-----
['僕のワンダフル・ジャーニー']
-仮面ライダー平成ジェネレーションズ FINAL ビルド&エグゼイドwithレジェンドライダー----

In [24]:
movie_series_ja=sorted(list(set(movie_series_ja)))
movie_series_ja

[" Fatestay night Heaven's Feel I. presage flower",
 " Fatestay night Heaven's Feel II. lost butterfly",
 ' Free! Timeless Medley 約束',
 ' Free! Timeless Medley 絆',
 ' 仮面ライダーゴースト 100の眼魂とゴースト運命の瞬間',
 ' 妖怪ウォッチ 空飛ぶクジラとダブル世界の大冒険だニャン!',
 ' 妖怪ウォッチ 誕生の秘密だニャン！',
 '64 ロクヨン 前編',
 '64 ロクヨン 後編',
 'ANEMONE／交響詩篇エウレカセブン ハイエボリューション',
 'DTC 湯けむり純情篇 from HiGH&LOW',
 'GODZILLA 怪獣惑星',
 'GODZILLA 星を喰う者',
 'HiGH&LOW THE MOVIE',
 'HiGH&LOW THE MOVIE2  END OF SKY',
 'HiGH&LOW THE RED RAIN',
 'HiGH&LOW THE WORST',
 'IT／イット THE END “それ”が見えたら、終わり。',
 'IT／イット “それ”が見えたら、終わり。',
 'ONE PIECE FILM GOLD',
 'ONE PIECE STAMPEDE',
 'PRINCE OF LEGEND',
 'ROAD TO HiGH&LOW',
 'Reゼロから始める異世界生活 Memory Snow',
 'Reゼロから始める異世界生活 氷結の絆',
 'T2 トレインスポッティング',
 'TAXi ダイヤモンド・ミッション',
 'honey',
 'orange',
 'それいけ！アンパンマン おもちゃの星のナンダとルンダ',
 'それいけ！アンパンマン かがやけ！クルンといのちの星',
 'それいけ！アンパンマン きらめけ！アイスの国のバニラ姫',
 'それいけ！アンパンマン ブルブルの宝探し大冒険',
 'ちはやふる ー結びー',
 'ちはやふる 上の句',
 'ちはやふる 下の句',
 'アサシン クリード',
 'アス',
 'アベンジャーズ／インフィニティ・ウォー',
 'アベンジャーズ／エンドゲーム',
 'アリス・イン・ワン

In [61]:
movie_series_ja

[" Fatestay night Heaven's Feel I. presage flower",
 " Fatestay night Heaven's Feel II. lost butterfly",
 ' Free! Timeless Medley 約束',
 ' Free! Timeless Medley 絆',
 ' 仮面ライダーゴースト 100の眼魂とゴースト運命の瞬間',
 ' 妖怪ウォッチ 空飛ぶクジラとダブル世界の大冒険だニャン!',
 ' 妖怪ウォッチ 誕生の秘密だニャン！',
 '64 ロクヨン 前編',
 '64 ロクヨン 後編',
 'ANEMONE／交響詩篇エウレカセブン ハイエボリューション',
 'DTC 湯けむり純情篇 from HiGH&LOW',
 'GODZILLA 怪獣惑星',
 'GODZILLA 星を喰う者',
 'HiGH&LOW THE MOVIE',
 'HiGH&LOW THE MOVIE2  END OF SKY',
 'HiGH&LOW THE RED RAIN',
 'HiGH&LOW THE WORST',
 'IT／イット THE END “それ”が見えたら、終わり。',
 'IT／イット “それ”が見えたら、終わり。',
 'ONE PIECE FILM GOLD',
 'ONE PIECE STAMPEDE',
 'PRINCE OF LEGEND',
 'ROAD TO HiGH&LOW',
 'Reゼロから始める異世界生活 Memory Snow',
 'Reゼロから始める異世界生活 氷結の絆',
 'T2 トレインスポッティング',
 'TAXi ダイヤモンド・ミッション',
 'honey',
 'orange',
 'それいけ！アンパンマン おもちゃの星のナンダとルンダ',
 'それいけ！アンパンマン かがやけ！クルンといのちの星',
 'それいけ！アンパンマン きらめけ！アイスの国のバニラ姫',
 'それいけ！アンパンマン ブルブルの宝探し大冒険',
 'ちはやふる ー結びー',
 'ちはやふる 上の句',
 'ちはやふる 下の句',
 'アサシン クリード',
 'アス',
 'アベンジャーズ／インフィニティ・ウォー',
 'アベンジャーズ／エンドゲーム',
 'アリス・イン・ワン

In [79]:
import re 
#数字の除去
p = re.compile('[^0123456789]+')
#ローマ数字
q = re.compile('[\u2160-\u217F]+')

word_counter = {}
for title in movie_series_ja:
  for word in title.split(' '):
    word = re.sub(r'\d+' ,'' ,word)
    word = re.sub(q,'' ,word)
    word = re.sub(r'[I,II,V]', '', word)
    word = word.replace('.','')
    if '／' in word:
      word = word.split('／')[0]
    if word != 'THE':
      # dict.get(key, 0)はdictにkeyがあればdict[key]を、なければ0を返すメソッドです
      word_counter[word] = word_counter.get(word, 0) + 1

In [80]:
w1 =  'I II 熱血篇 傷物語V 冷血篇'
q = re.compile('[\u2160-\u217F]+')
w = re.sub(r'[\u2160-\u217F]+', '', w1)
w = re.sub(r'[I,II,V]', '', w1)
w

'  熱血篇 傷物語 冷血篇'

In [81]:
#word_counter = sorted(word_counter.items())
#sortedで返ってくるのはリスト型なため辞書型に直す
word_counter_={}
for k, v in sorted(word_counter.items(), key=lambda x: -x[1]):
  word_counter_.update({k:v})

word_counter_.pop('')
word_counter = pd.Series(word_counter_)

In [82]:
#2回以上カウントされるものを抽出
word_counter = word_counter[word_counter>1]
word_counter

MOE                  7
HiGH&LOW             6
それいけ！アンパンマン          4
ドラえもん                4
名探偵コナン               4
機動戦士ガンダム             4
ORGN                 4
T                    3
ちはやふる                3
スパイダーマン              3
傷物語                  3
宇宙戦艦ヤマト              3
愛の戦士たち               3
Fatestay             2
night                2
Heaven's             2
Feel                 2
Free!                2
Timeless             2
Medley               2
妖怪ウォッチ               2
ロクヨン                 2
ハイエボリューション           2
GODZLLA              2
END                  2
OF                   2
“それ”が見えたら、終わり。       2
ONE                  2
PECE                 2
Reゼロから始める異世界生活       2
アベンジャーズ              2
ガールズ＆パンツァー           2
クレヨンしんちゃん            2
コードギアス               2
スター・ウォーズ             2
デッドプール               2
パディントン               2
ファンタスティック・ビースト       2
ポケットモンスター            2
モンスターストライク           2
モンスター・ホテル            2
仮面ライダー平成ジェネレーションズ    2
僕のヒーローアカデミア          2
家族はつらいよ    

In [90]:
#仮面ライダーは別途追加
word_counter['仮面ライダー']=2

In [91]:
def boolean_contains_keyword(input_df:pd.DataFrame)->pd.DataFrame:
  output_df = input_df.copy()
  cols=[]
  for keyword in word_counter.index:
    output_df[f'Title(ja)_contains_{keyword}']=output_df['Title(ja)'].dropna().apply(lambda x: 1 if keyword in x else 0)
    cols.append(f'Title(ja)_contains_{keyword}')
  return output_df[cols]

In [92]:
df = pd.concat([train,test], axis=0).reset_index(drop=True)
title_df = boolean_contains_keyword(df)
title_df.sum(axis=0)

Title(ja)_contains_MOE                   0.0
Title(ja)_contains_HiGH&LOW              6.0
Title(ja)_contains_それいけ！アンパンマン           4.0
Title(ja)_contains_ドラえもん                 4.0
Title(ja)_contains_名探偵コナン                7.0
Title(ja)_contains_機動戦士ガンダム              5.0
Title(ja)_contains_ORGN                  0.0
Title(ja)_contains_T                    46.0
Title(ja)_contains_ちはやふる                 3.0
Title(ja)_contains_スパイダーマン               3.0
Title(ja)_contains_傷物語                   3.0
Title(ja)_contains_宇宙戦艦ヤマト               3.0
Title(ja)_contains_愛の戦士たち                3.0
Title(ja)_contains_Fatestay              0.0
Title(ja)_contains_night                 2.0
Title(ja)_contains_Heaven's              2.0
Title(ja)_contains_Feel                  2.0
Title(ja)_contains_Free!                 5.0
Title(ja)_contains_Timeless              2.0
Title(ja)_contains_Medley                2.0
Title(ja)_contains_妖怪ウォッチ               13.0
Title(ja)_contains_ロクヨン                  2.0
Title(ja)_

#データの結合

In [93]:
train.shape, test.shape, df.shape

((642, 9017), (167, 9017), (809, 9017))

In [94]:
ALL_DATA = pd.concat([df, title_df], axis=1)
train_df = ALL_DATA[0:len(train)].reset_index(drop=True)
test_df = ALL_DATA[len(train):].reset_index(drop=True)

train_df.shape, test_df.shape

((642, 9063), (167, 9063))

In [95]:
train_df.to_csv('/content/drive/MyDrive/Movies/preprocessed/train_2016-2018_title.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/Movies/preprocessed/test_2019_title.csv', index=False)